<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristian Oyarzo M.
- Nombre de alumno 2: Sebastián Quenti A.


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/sebaquenti/Repositorio-MDS7202)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y', dayfirst=True)
df.head()


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [2]:
from sklearn import set_config
set_config(transform_output="pandas")
from sklearn.model_selection import train_test_split


# Parte 1
X_train, temp_data = train_test_split(df, test_size=0.3, random_state=1323)
X_val, X_test = train_test_split(temp_data, test_size=1/3, random_state=1323)

# Separar las características (X) y la variable objetivo (y)
y_train = X_train['quantity']
y_val = X_val['quantity']
y_test = X_test['quantity']

X_train = X_train.drop(columns=['quantity'])
X_val = X_val.drop(columns=['quantity'])
X_test = X_test.drop(columns=['quantity'])


In [3]:
# Parte 2
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

def extract_date_features(X):
    X = X.copy()
    X['day'] = X['date'].dt.day.astype('category')
    X['month'] = X['date'].dt.month.astype('category')
    X['year'] = X['date'].dt.year.astype('category')
    return X

# FunctionTransformer
date_transformer = FunctionTransformer(extract_date_features)

In [4]:
df.dtypes

id                    int64
date         datetime64[ns]
city                 object
lat                 float64
long                float64
pop                   int64
shop                 object
brand                object
container            object
capacity             object
price               float64
quantity              int64
dtype: object

In [5]:
# Parte 3
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Definir las columnas categóricas y numéricas
categorical_cols = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']
numerical_cols = ['lat', 'long', 'pop', 'price']

# Crear un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(sparse_output=False), categorical_cols)
    ]
)

preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['lat', 'long', 'pop', 'price']),
                                ('cat', OneHotEncoder(sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity', 'day', 'month', 'year'])])

In [6]:
# Parte 4
from sklearn.dummy import DummyRegressor

# Crear un Pipeline
pipeline = Pipeline(steps=[
    ('date_extractor', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean')) 
])

In [7]:
# Parte 5
from sklearn.metrics import mean_absolute_error

# Ajustar el pipeline con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Hacer predicciones en el conjunto de validación
y_pred_val = pipeline.predict(X_val)

# Calcular el mean_absolute_error (MAE) en los datos de validación
mae = mean_absolute_error(y_val, y_pred_val)

# Reportar el MAE
print(f"Mean Absolute Error en los datos de validación: {mae}")

Mean Absolute Error en los datos de validación: 13054.420647921506


El Mean Absolute Error de 13.054,42 unidades nos dice que, en promedio, el modelo se está equivocando por esa cantidad de ventas. Si consideramos que este modelo solo predice el promedio, es normal que no sea muy preciso. Cabe mencionar que la variable "quantity" no se escaló por ambiguedad de enunciado y para facilitar la interpretabilidad de los resultados.

In [8]:
# Parte 6
from xgboost import XGBRegressor

# Pipeline
pipeline2 = Pipeline(steps=[
    ('date_extractor', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor()) 
])

# Ajustar el pipeline con los datos de entrenamiento
pipeline2.fit(X_train, y_train)

# Hacer predicciones en el conjunto de validación
y_pred_val = pipeline2.predict(X_val)

# Calcular el mean_absolute_error (MAE) en los datos de validación
mae = mean_absolute_error(y_val, y_pred_val)

# Reportar el MAE
print(f"Mean Absolute Error en los datos de validación: {mae}")

Mean Absolute Error en los datos de validación: 2600.4661273303886



El MAE con el XGBRegressor es de 2.600,47, lo que muestra una mejora considerable en comparación con el DummyRegressor, cuyo MAE fue de 13.054. Esto significa que el XGBRegressor es mucho más preciso al predecir las ventas, reduciendo el error en casi un 80%. Por lo tanto, el XGBRegressor es claramente mejor para este problema, ya que captura patrones más complejos en los datos y permite tomar decisiones más acertadas en cuanto a la planificación de inventario y producción.


In [9]:
import pickle

# Guardar el modelo DummyRegressor
with open('dummy_regressor.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

# Guardar el modelo XGBRegressor
with open('xgb_regressor.pkl', 'wb') as f:
    pickle.dump(pipeline2, f)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [10]:
# Parte 1 y 2
# Obtener los nombres de las características
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# Definir las restricciones de monotonía
monotonic_constraints = {"num__price": -1}  # Monotonía negativa para 'price'

# Crear el modelo XGBRegressor con la restricción de monotonicidad
xgb_model = XGBRegressor(monotone_constraints=monotonic_constraints)

# Crear el pipeline final que incluye el modelo XGBRegressor
pipeline_with_model = Pipeline(steps=[
    ('date_extractor', date_transformer), 
    ('preprocessor', preprocessor),
    ('regressor', xgb_model) 
])

# Entrenar el pipeline con la restricción
pipeline_with_model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de validación
y_pred_val = pipeline_with_model.predict(X_val)

# Calcular el mean_absolute_error (MAE) en los datos de validación
mae = mean_absolute_error(y_val, y_pred_val)

# Reportar el MAE
print(f"Mean Absolute Error con restricción de monotonicidad en precio: {mae}")

Mean Absolute Error con restricción de monotonicidad en precio: 2542.1669387254515


3. El MAE antes era de 2600.466, ahora, al incluir esta relación, baja a 2542.167. Se tiene que mejoró el modelo, aunque el cambio en el MAE fue pequeño, esto indica que, si bien el precio tiene un impacto en la demanda, puede que existan otros factores importantes que también influencian la predicción.

In [11]:
# Parte 4
with open('monotonic_xgb_model.pkl', 'wb') as f:
    pickle.dump(pipeline_with_model, f)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
  5. Guardar su modelo en un archivo .pkl [1 punto]

In [12]:
X_train.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price
6786,6890,2018-05-31,Athens,37.96245,23.68708,664046,shop_3,kinder-cola,can,330ml,0.80
2897,2940,2014-09-30,Athens,37.96245,23.68708,668203,shop_3,lemon-boost,glass,500ml,0.92
1947,1977,2013-10-31,Patra,38.24444,21.73444,166301,shop_6,orange-power,plastic,1.5lt,1.67
3607,3661,2015-05-31,Larisa,39.63689,22.41761,141732,shop_5,kinder-cola,glass,500ml,1.48
4723,4795,2016-06-30,Irakleion,35.32787,25.14341,137302,shop_2,lemon-boost,plastic,1.5lt,2.00


In [13]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Optimizar también el min_frequency de OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Preprocesamiento de las variables
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_cols)
        ]
    )
    
    # Definir las restricciones de monotonía para 'price'
    monotonic_constraints = {"num__price": -1}

    # Crear el modelo XGBRegressor con los hiperparámetros sugeridos por Optuna
    xgb_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints=monotonic_constraints,
        random_state=1323,
        n_jobs=-1
    )

    # Crear el pipeline
    pipeline = Pipeline(steps=[
        ('date_extractor', FunctionTransformer(extract_date_features)),
        ('preprocessor', preprocessor),
        ('regressor', xgb_model)
    ])

    # Entrenar el pipeline
    pipeline.fit(X_train, y_train)

    # Hacer predicciones en el conjunto de validación
    y_pred_val = pipeline.predict(X_val)

    # Calcular el MAE
    mae = mean_absolute_error(y_val, y_pred_val)

    # Almacenar el mejor pipeline entrenado
    trial.set_user_attr('best_pipeline', pipeline)

    return mae


# Crear el estudio para optimizar los hiperparámetros con TPESampler
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=1323))

# Ejecutar la optimización
study.optimize(objective, timeout=300)

# Mostrar los mejores hiperparámetros y el mejor valor de MAE
print(f"Mejores hiperparámetros: {study.best_params}")
print(f"Mejor MAE: {study.best_value}")

# Recuperar el mejor pipeline
best_pipeline = study.best_trial.user_attrs['best_pipeline']

Mejores hiperparámetros: {'learning_rate': 0.09008926211603463, 'n_estimators': 879, 'max_depth': 7, 'max_leaves': 62, 'min_child_weight': 5, 'reg_alpha': 0.6181435202067234, 'reg_lambda': 0.6838731956743429, 'min_frequency': 0.048879903006237665}
Mejor MAE: 2115.683518428278


In [14]:
print(f"Número de iteraciones: {study.trials_dataframe().shape[0]}")

Número de iteraciones: 33


3. La optimización con Optuna mejoró significativamente el rendimiento del modelo, reduciendo el MAE de 2542 a 2115.68 en solo 34 iteraciones dentro de un límite de 5 minutos. Los nuevos hiperparámetros, como un learning_rate de 0.09 y 879 estimadores, permitieron al modelo ajustarse mejor a los datos. Este resultado demuestra que la optimización de hiperparámetros mediante métodos bayesianos puede mejorar sustancialmente la precisión del modelo, logrando configuraciones óptimas que un ajuste manual no capturaría.

4. 
- leraning_rate: Controla el tamaño de los pasos que da el modelo en cada iteración de boosting. Un valor bajo hace que el modelo aprenda más lentamente, lo que generalmente mejora la precisión pero aumenta el tiempo de entrenamiento. Un valor alto puede hacer que el modelo converja más rápido, pero con el riesgo de sobreajuste. El rango es adecuado, ya que valores muy bajos permiten una convergencia estable, mientras que valores altos acelerarían el aprendizaje inicial. 
- n_estimators: Define el número total de árboles que se generarán en el proceso de boosting. Más árboles generalmente mejoran la capacidad del modelo para ajustarse a los datos, pero a costa de un mayor tiempo de entrenamiento. El rango es adecuado, ya que un número bajo de estimadores puede ser insuficiente para capturar patrones complejos, mientras que un número muy alto puede llevar a un sobreajuste o tiempos de entrenamiento innecesariamente largos.
- max_depth: Limita la profundidad máxima de cada árbol. Un árbol más profundo puede capturar interacciones complejas entre características, pero también aumenta el riesgo de sobreajuste. El rango es adecuado. Profundidades menores pueden evitar el sobreajuste, mientras que profundidades más altas permiten al modelo aprender interacciones más complejas.
- max_leaves: Controla el número máximo de hojas por árbol. Un mayor número de hojas permite capturar más divisiones dentro de cada árbol, mejorando la precisión, pero también puede aumentar el riesgo de sobreajuste. El rango es razonable. Valores bajos ayudan a simplificar el modelo y evitar sobreajustes, mientras que valores más altos permiten capturar más detalles en los datos.
- min_child_weight: Especifica el peso mínimo que debe tener una hoja para que se considere válida. Un valor más alto regulariza el modelo, evitando que se creen hojas a partir de muy pocos datos. El rango es adecuado para controlar el crecimiento excesivo de los árboles y prevenir sobreajustes. Un valor bajo permitiría más divisiones, mientras que valores altos incrementan la regularización.
- reg_alpha (regularización L1): Penaliza el uso de características innecesarias mediante regularización L1, lo que ayuda a reducir la complejidad del modelo y evitar el sobreajuste. El rango es adecuado. Valores cercanos a 0 permiten más flexibilidad, mientras que valores cercanos a 1 aplican una fuerte regularización, útil para reducir el sobreajuste.
- reg_lambda (regularización L2): Esta penalización reduce la magnitud de los coeficientes del modelo, haciendo que el modelo sea menos sensible a las características menos importantes. El rango es también adecuado, ya que permite ajustar la fuerza de la regularización. Un valor más alto impone mayor penalización, ayudando a controlar el sobreajuste.
- min_frequency: Define la frecuencia mínima que debe tener una categoría para ser codificada como una característica. Las categorías menos frecuentes pueden ser agrupadas. El rango es adecuado, ya que permite experimentar con una alta granularidad o con una mayor generalización al agrupar categorías poco frecuentes.


In [15]:
# Parte 5
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_pipeline, f)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

1. Pruning es una técnica que detiene de forma anticipada el entrenamiento de modelos cuando se identifica que su rendimiento intermedio no es prometedor. Esto optimiza el uso de recursos y tiempo, ya que evita gastar computación en intentos que probablemente no superarán a los anteriores. En resumen, pruning acelera la búsqueda de los mejores hiperparámetros al concentrar el entrenamiento en los modelos más prometedores, mejorando la eficiencia del proceso de optimización.

In [16]:
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb

optuna.logging.set_verbosity(optuna.logging.WARNING)

# Parte 2, 3 y 4
# Definir el objective function con Pruning
def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Optimizar también el min_frequency de OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_cols)
        ]
    )

    # Crear el pipeline para el preprocesamiento y extracción de características de fecha
    pipeline = Pipeline(steps=[
        ('date_extractor', FunctionTransformer(extract_date_features)),
        ('preprocessor', preprocessor)
    ])
    
    # Aplicar el pipeline de preprocesamiento a los datos de entrenamiento y validación
    X_train_preprocessed = pipeline.fit_transform(X_train)
    X_val_preprocessed = pipeline.transform(X_val)
    
    # Crear el DMatrix para XGBoost (API nativa)
    dtrain = xgb.DMatrix(X_train_preprocessed, label=y_train)
    dval = xgb.DMatrix(X_val_preprocessed, label=y_val)

    params = {
        "objective": "reg:squarederror",
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "max_leaves": max_leaves,
        "min_child_weight": min_child_weight,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "monotone_constraints": (0, -1),
        "eval_metric": "mae"
    }

    # Entrenar el modelo con el pruning callback
    pruning_callback = XGBoostPruningCallback(trial, "validation-mae")
    
    model = xgb.train(
        params,
        dtrain,
        evals=[(dval, "validation")],
        num_boost_round=n_estimators,
        early_stopping_rounds=50,
        callbacks=[pruning_callback],
        verbose_eval=False  # Para silenciar la salida de XGBoost
    )

    # Hacer predicciones en el conjunto de validación
    y_pred_val = model.predict(dval)

    # Calcular el MAE
    mae = mean_absolute_error(y_val, y_pred_val)

    # Almacenar el mejor modelo entrenado
    trial.set_user_attr('best_model', model)

    return mae

# Crear el estudio para optimizar los hiperparámetros con TPESampler
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=1323))

# Ejecutar la optimización con límite de tiempo de 5 minutos y mostrar la barra de progreso
study.optimize(objective, timeout=300, show_progress_bar=True)

# Mostrar los mejores hiperparámetros y el mejor valor de MAE
print(f"Mejores hiperparámetros: {study.best_params}")
print(f"Mejor MAE: {study.best_value}")

# Reportar el número de trials realizados
n_trials = study.trials_dataframe().shape[0]
print(f"Número de trials realizados: {n_trials}")

# Guardar el mejor modelo entrenado
best_model = study.best_trial.user_attrs['best_model']



# Parte 5
# Guardar el modelo en un archivo .pkl
import pickle
with open('pruned_xgb_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

   0%|          | 00:00/05:00

Mejores hiperparámetros: {'learning_rate': 0.04514866986126306, 'n_estimators': 848, 'max_depth': 8, 'max_leaves': 86, 'min_child_weight': 5, 'reg_alpha': 0.34513989327721983, 'reg_lambda': 0.5993780942418381, 'min_frequency': 0.026409350621963146}
Mejor MAE: 2076.9981544540683
Número de trials realizados: 97


4.  Se realizaron 96 trials, dio un MAE de 2076.998 y los hiperparámetros, 'learning_rate': 0.045, 'n_estimators': 848, 'max_depth': 8, 'max_leaves': 86, 'min_child_weight': 5, 'reg_alpha': 0.345, 'reg_lambda': 0.599, 'min_frequency': 0.0264.

    El MAE representa una pequeña disminución respecto al anterior, este cambio sugiere que la implementación del pruning ha permitido ajustar mejor los hiperparámetros del modelo, identificando combinaciones que generan predicciones más precisas. La mejora puede deberse a que el pruning detiene de manera anticipada los intentos de entrenamiento no prometedores, lo que libera recursos y tiempo para explorar configuraciones más efectivas. Esto permite que el proceso de optimización se enfoque en las mejores configuraciones posibles, resultando en un modelo más preciso.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

Esta parte de hará respecto al último entrenamiento realizado

In [17]:
# Parte 1
import optuna.visualization as vis

fig = vis.plot_optimization_history(study)
fig.show()


In [18]:
# Parte 2
fig = vis.plot_parallel_coordinate(study)
fig.show()

In [19]:
# Parte 3
fig = vis.plot_param_importances(study)
fig.show()

4. Desde el trial 65 se empiezan a notar una mejora considerable, ya que el MAE se reduce a la magnitud de 2000 de manera estable. Se puede observar también que en la iteración 6 se logra llegar a ese nivel de MAE, pero esa reducción no se mantiene constante.
5. Para obtener mejores resultados, se observa que los valores bajos de min_frequency (cercanos a 0.1 o menores) están relacionados con menores errores, lo que sugiere que mantener más categorías en las variables categóricas puede ser beneficioso. El max_depth entre 6 y 8 y max_leaves superiores a 60 muestran buenas tendencias en términos de minimizar el MAE. Un min_child_weight cercano a 1 también parece mejorar los resultados.
6. Claramente, el hiperparámetro más importante en la optimización del modelo es min_frequency, siendo incluso diez veces más relevante que el segundo que es reg_lambda.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [21]:
# Parte 1
mae_data = {
    'Modelo': [
        'Baseline (Dummy Regressor)',
        'XGBoost Básico',
        'XGBoost con Monotonicidad',
        'XGBoost con Optuna',
        'XGBoost con Optuna y Pruning'
    ],
    'MAE Validación': [
        13054.420647921506,  
        2600.4661273303886,  
        2542.1669387254515,
        2115.683518428278,
        2076.9981544540683]
}

# Crear el DataFrame con los datos simulados
mae_summary_df = pd.DataFrame(mae_data)

# Mostrar la tabla resumen del MAE
mae_summary_df

,Modelo,MAE Validación
0,Baseline (Dummy Regressor),13054.420648
1,XGBoost Básico,2600.466127
2,XGBoost con Monotonicidad,2542.166939
3,XGBoost con Optuna,2115.683518
4,XGBoost con Optuna y Pruning,2076.998154


2. El modelo con el mejor rendimiento es XGBoost con Optuna y Pruning, ya que obtuvo el menor valor de MAE (2076.998) en la validación. Esto indica que la combinación de optimización de hiperparámetros mediante Optuna y el uso de pruning para detener de manera temprana los entrenamientos no prometedores permitió ajustar mejor los parámetros del modelo, obteniendo predicciones más precisas comparadas con las otras versiones del modelo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>